# clustering notes and documents based on tags 
for a while i have been using markdown to take notes and tags are good way to sort and find related documents.
clustering notes based on tags could be usefull way to  find and review related pieces of information in your library of notes.

## vectorisation and embeding
to efectively cluster documentts based on tags we could go with few aproach. 
- create giant booolean vector where 1 means that document has that tag, and 0 means it doesnt.

this aproach can work but it doesnt take into acount that some tags are related. for example document with the tags:[#math] and [#python] would be as far apart(norm/distance) as [#cpp] and [#foodrecipes]. math and python are clearly more related to each other but this model wouldnt take it into account.

- create semantic embedings of the tags based on what topics they represent and vector of the documment woudl be mean of the tags it has
this aproach needs little more work, but solves the problem of related tags. for the sake of demonstration im using handwriten weights/vector embedings of few predeterminet tags though there are algoriths/ libraries that woudl make embeding out of any tag.
[math, coding, art, fantasy, gaming, hobby] this will be semantinc axis that I woudl use for each tag
exmple
    "NLA":   [0.90, 0.20, 0.00, 0.00, 0.00, 0.00] since its mainly about math but also we learn numerical methods for cooding

  


## Code and explanations

In [2]:
#importing libraries
import numpy as np

### norms 
few norms for testing
infinity norm is interesting as it will since its searching for maximum axis that woudl mean it would only take main topic of the document and igonore any other subtopic on semantic axis

In [3]:
def norm2 (a): return np.sqrt((a ** 2).sum(axis = -1))
def norm1 (a): return np.abs(a).sum(axis=-1)
def normifn (a): return np.max(np.abs(a), axis = -1)

### Density-based clustering (DBSCAN)
Switching from agglomerative merging to DBSCAN so we don't need to pre-pick the number of clusters.
- eps controls how close tag-embedding vectors must be to be neighbors.
- min_samples sets how many neighbors (including the point itself) are needed to form a dense core.
- Points that never reach that density become noise with label -1.


In [4]:
def pairwise_distances(points, norm):
    n = len(points)
    d = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            dist = norm(points[i] - points[j])
            d[i, j] = d[j, i] = dist
    return d

def dbscan(dist_matrix, eps, min_samples):
    n = dist_matrix.shape[0]
    labels = -np.ones(n, dtype=int)
    visited = np.zeros(n, dtype=bool)
    cluster_id = 0

    for i in range(n):
        if visited[i]:
            continue
        visited[i] = True
        neighbors = np.where(dist_matrix[i] <= eps)[0]
        if neighbors.size < min_samples:
            labels[i] = -1
            continue

        labels[i] = cluster_id
        seeds = [p for p in neighbors if p != i]

        while seeds:
            point = seeds.pop()
            if not visited[point]:
                visited[point] = True
                point_neighbors = np.where(dist_matrix[point] <= eps)[0]
                if point_neighbors.size >= min_samples:
                    for nb in point_neighbors:
                        if nb not in seeds:
                            seeds.append(nb)
            if labels[point] == -1:
                labels[point] = cluster_id
            elif labels[point] != cluster_id:
                labels[point] = cluster_id
        cluster_id += 1

    return labels


### Distance matrix helper
We pre-compute pairwise distances between document embeddings once and reuse them in DBSCAN.


In [5]:
def summarize_clusters(labels, names):
    groups = {}
    for idx, label in enumerate(labels):
        groups.setdefault(label, []).append(names[idx])
    return groups


### sample data and sample tag vectors

In [6]:
# semantic axes:
#[math, coding, art, fantasy, gaming, hobby]
tag_vectors = {
    "math":            np.array([1.00, 0.00, 0.00, 0.00, 0.00, 0.00]),
    "NLA":             np.array([0.90, 0.20, 0.00, 0.00, 0.00, 0.00]),
    "calculus":        np.array([0.85, 0.00, 0.00, 0.00, 0.00, 0.00]),
    "analysis":        np.array([0.80, 0.00, 0.00, 0.00, 0.00, 0.00]),

    "coding":          np.array([0.00, 1.00, 0.00, 0.00, 0.00, 0.00]),
    "python":          np.array([0.00, 0.90, 0.00, 0.00, 0.00, 0.00]),
    "cpp":             np.array([0.00, 0.80, 0.00, 0.00, 0.00, 0.00]),
    "algorithms":      np.array([0.10, 0.85, 0.00, 0.00, 0.00, 0.00]),
    "project":         np.array([0.00, 0.70, 0.00, 0.00, 0.00, 0.30]),  

    "art":             np.array([0.00, 0.00, 1.00, 0.00, 0.00, 0.00]),
    "design":          np.array([0.00, 0.00, 0.85, 0.00, 0.00, 0.00]),
    "drawing":         np.array([0.00, 0.00, 0.90, 0.00, 0.00, 0.30]),

    "fantasy":         np.array([0.00, 0.00, 0.00, 1.00, 0.00, 0.00]),
    "worldbuilding":   np.array([0.00, 0.00, 0.00, 0.90, 0.00, 0.30]),
    "lore":            np.array([0.00, 0.00, 0.00, 0.85, 0.00, 0.00]),
    "dnd":             np.array([0.00, 0.00, 0.00, 0.80, 0.30, 0.30]),  

    "gaming":          np.array([0.00, 0.00, 0.00, 0.00, 1.00, 0.00]),
    "game-dev":        np.array([0.00, 0.50, 0.00, 0.00, 0.90, 0.00]),
    "helldivers":      np.array([0.00, 0.00, 0.00, 0.00, 0.95, 0.00]),

    "hobby":           np.array([0.00, 0.00, 0.00, 0.00, 0.00, 1.00]),
    "journal":         np.array([0.00, 0.00, 0.00, 0.00, 0.00, 0.90]),
    "recipes":         np.array([0.00, 0.00, 0.00, 0.00, 0.00, 0.80])
}

In [7]:
#AI generated sample data
documents = {
    # Pure Math -----
    "la_notes":              ["math", "linear-algebra"],
    "calc_homework":         ["math", "calculus"],
    "analysis_summary":      ["math", "analysis"],
    "probability_notes":     ["math"],

    # ----- Pure Coding -----
    "python_basics":         ["coding", "python"],
    "cpp_optimizations":     ["coding", "cpp"],
    "algo_design":           ["coding", "algorithms"],
    "project_todo":          ["coding", "project"],

    # ----- Pure Art -----
    "portrait_sketch":       ["art", "drawing"],
    "ui_design_draft":       ["art", "design"],
    "color_theory":          ["art"],

    # ----- Pure Fantasy -----
    "dnd_session_1":         ["dnd", "worldbuilding"],
    "world_map":             ["fantasy", "worldbuilding"],
    "ancient_lore":          ["fantasy", "lore"],
    "magic_system":          ["fantasy"],

    # ----- Pure Gaming -----
    "game_review_hd2":       ["gaming", "helldivers"],
    "build_guide_slayer":    ["gaming"],
    "game_dev_plan":         ["gaming", "game-dev", "project"],

    # ----- Pure Hobby -----
    "daily_journal":         ["hobby", "journal"],
    "recipe_tacos":          ["hobby", "recipes"],
    "fitness_log":           ["hobby"],

    # ----- Math + Coding Mix -----
    "math_for_algos":        ["math", "algorithms"],
    "python_numerics":       ["coding", "math", "python"],
    "machine_learning_intro":["math", "coding", "algorithms", "python"],
    "project_summary":       ["coding", "math", "project"],

    # ----- Coding + Fantasy Mix -----
    "python_for_dnd":        ["coding", "dnd"],
    "campaign_tracker_tool": ["coding", "fantasy", "project"],
    "map_generator_code":    ["coding", "worldbuilding", "project"],
    "spell_calc_program":    ["coding", "fantasy", "math"],

    # ----- Art + Fantasy Mix -----
    "character_concepts":    ["art", "fantasy"],
    "world_map_art":         ["art", "worldbuilding"],
    "magic_creature_design": ["art", "fantasy", "design"],
    "ui_for_fantasy_game":   ["art", "gaming", "design"],

    # ----- Coding + Gaming Mix -----
    "ai_bot_for_games":      ["coding", "gaming", "algorithms"],
    "game_modding_notes":    ["gaming", "coding"],
    "game_dev_diary":        ["gaming", "project", "journal"],
    "procedural_generation": ["coding", "game-dev", "fantasy"],

    # ----- Weird Crossovers (stress tests) -----
    "math_of_color":         ["math", "art"],
    "fantasy_statistics":    ["math", "fantasy"],
    "cooking_for_dnd_party": ["hobby", "fantasy", "recipes"],
    "art_of_algorithms":     ["math", "coding", "art"],
    "lore_based_ai":         ["coding", "fantasy", "algorithms"],
    "game_study_notebook":   ["math", "gaming", "journal"],
    "dnd_budget_sheet":      ["math", "hobby", "dnd"],
    "fantasy_cooking":       ["fantasy", "recipes"],
}


In [8]:
def document_vector(tags):
    """Average vector of a set of tags."""
    vecs = [tag_vectors[tag] for tag in tags if tag in tag_vectors]
    if not vecs:
        return np.zeros(6)
    return np.mean(vecs, axis=0)
    
# converting docs to embeddings
doc_embeddings = {name: document_vector(tags) for name, tags in documents.items()}
names = list(doc_embeddings.keys())
array = np.vstack([vector for vector in doc_embeddings.values()])

# Distance matrix + DBSCAN parameters
dist_matrix = pairwise_distances(array, norm2)
EPS = np.median(dist_matrix[np.triu_indices_from(dist_matrix, k=1)])  # start with median distance
MIN_SAMPLES = 3

labels = dbscan(dist_matrix, eps=EPS, min_samples=MIN_SAMPLES)
clusters = summarize_clusters(labels, names)

print(f"DBSCAN eps={EPS:.3f}, min_samples={MIN_SAMPLES}")
for cid, docs in sorted(clusters.items()):
    label = "Noise" if cid == -1 else f"Cluster {cid}"
    print(f"{label}: {docs}")


{'build_guide_slayer', 'game_review_hd2'}


{'la_notes', 'probability_notes', 'calc_homework', 'analysis_summary'}


{'world_map_art', 'character_concepts', 'magic_creature_design'}


{'algo_design', 'math_for_algos', 'python_basics', 'game_dev_plan', 'art_of_algorithms', 'cpp_optimizations', 'spell_calc_program', 'ai_bot_for_games', 'procedural_generation', 'fantasy_statistics', 'project_todo', 'project_summary', 'campaign_tracker_tool', 'map_generator_code', 'python_for_dnd', 'game_modding_notes', 'python_numerics', 'math_of_color', 'machine_learning_intro', 'lore_based_ai'}


{'game_study_notebook', 'daily_journal', 'recipe_tacos', 'world_map', 'fitness_log', 'dnd_session_1', 'dnd_budget_sheet', 'game_dev_diary', 'magic_system', 'cooking_for_dnd_party', 'fantasy_cooking', 'ancient_lore'}


{'ui_design_draft', 'color_theory', 'ui_for_fantasy_game', 'portrait_sketch'}


